# log mining script ICON 

In [155]:
import re
import sys
import logging
from datetime import datetime
from dataclasses import dataclass, fields
from typing import Optional, Iterable

import numpy as np


# My functions 

- Find the tables and save them in log_fileclass instances? 
- So this entails finding the pattern of the table, and cropping my document around it 
- If it is the small table, skip (use the code already there)
- Save values of the table into a csv or a json 
- if multiple tables, sum or take max? of value 8e.g., one value for integrate_nh #calls 
- Find the timestamp of the document 
- Add that to the json 
- find commit nb? 
- branch 


In [177]:
# Read file 
def read_logfile(filename):
    with open(filename, "r", encoding="latin-1") as f:
        # read file into list of lines, remove empty lines
        full_file = f.read()
        log_file = [e for e in full_file.split("\n") if e != ""]
    return log_file

log_file = read_logfile("LOG.check.mch_bench_r19b07_icon2.run.o")

In [13]:
# Create a dataclass to save each line of the table of interest 
@dataclass
class MatchResultsICON: 
    name: list = field(default_factory=list)
    call_nb: list = field(default_factory=list)
    t_min: list = field(default_factory=list)
    min_rank:list = field(default_factory=list)
    t_avg: list = field(default_factory=list)
    t_max: list = field(default_factory=list)
    max_rank: list = field(default_factory=list)
    total_min: list = field(default_factory=list)
    total_min_rank:list = field(default_factory=list)
    total_max: list = field(default_factory=list)
    total_max_rank: list = field(default_factory=list)
    total_avg: list = field(default_factory=list)
    pe: list = field(default_factory=list)

    # To make it subscriptable
    def __getitem__(self, key):
        return getattr(self, key)

    
    # def dict(self):
    #     return {k: str(v) for k, v in asdict(self).items()}

NameError: name 'dataclass' is not defined

In [127]:
import re

timing_start_regex = r"(?: +L? ?[a-zA-Z_.]+)"
timing_element_regex = r"(?:\[?\d+[.msh]?\d*s?\]? +)"
timing_regex = timing_start_regex + " +" + timing_element_regex + "{6,20} *(?!.)"
header_regex = r"name +.*calls.*"


In [128]:

class Table:
    def __init__(self, lines):
        # Remove white spaces at the beginning and end of each line
        self.lines = [line.strip() for line in lines]

    def __str__(self):
        return "\n".join(self.lines)

        


In [178]:

def isolate_table(log_file, header_regex, timing_regex):
    # Find header lines
    header_lines = [i for i, e in enumerate(log_file) if re.search(header_regex, e)]
    
    # find end line (footer)
    pattern = r'-'*165
    footer_lines = [i for i, e in enumerate(log_file) if re.search(pattern, e)]

    if not header_lines or not footer_lines:
        raise ValueError("No headers or footers found in the log")

    tables = []
    for i in range(len(header_lines)):
        start = header_lines[i]
        # Check if we have a corresponding footer for this header
        end = next((f for f in footer_lines if f > start), None)
        if end is None:
            break

        # Append table without headers
        table = Table(log_file[start+3:end])

        tables.append(table)

        # Remove this footer so it won't be used again
        footer_lines.remove(end)

    return tables


result = isolate_table(log_file, header_regex, timing_regex)
for table in result:
    print(table)


total                             4               53m11s   [2]              53m11s         53m11s   [0]            3191.350    [2]                  3191.350    [0]                  3191.350    4
L integrate_nh                   86400         0.12085s   [2]            0.13805s        1.4055s   [3]            2980.473    [0]                  2984.872    [3]                  2981.987    4
L nh_solve                    432000        0.00909s   [0]            0.00977s       0.13249s   [3]            1034.114    [0]                  1064.997    [3]                  1055.603    4
L nh_solve.veltend         518400        0.00127s   [2]            0.00130s       0.03737s   [2]             168.866    [1]                   169.421    [3]                   169.106    4
L nh_solve.cellcomp        864000        0.00032s   [1]            0.00051s       0.00243s   [1]             110.164    [1]                   110.621    [0]                   110.367    4
L nh_solve.edgecomp        864000        0.0

In [179]:

def remove_formatting(result):
    def process_line(line):
        line = re.sub('[\s]+', '-', line)
        if line.startswith("L"):
            line = line.lstrip("L-")
        return line.split('-')
    
    for table in result:
        table.lines = [process_line(line) for line in table.lines]
        

    return table


In [180]:
table = remove_formatting(result)

In [181]:
def store_lines(table):
    results_dataclass = MatchResultsICON()
    # List of attribute names in the order they should be accessed
    attribute_names = attributes = [field.name for field in fields(MatchResultsICON)]
    for i, attr_name in enumerate(attribute_names):
        for line in table.lines[1:]: 
            getattr(results_dataclass, attr_name).append(line[i])

    return results_dataclass

results_dataclass = store_lines(table)
print(results_dataclass)

MatchResultsICON(name=['integrate_nh', 'nh_solve', 'nh_solve.veltend', 'nh_solve.cellcomp', 'nh_solve.edgecomp', 'nh_solve.vnupd', 'nh_solve.vimpl', 'nh_solve.exch', 'nh_hdiff', 'transport', 'adv_horiz', 'adv_hflx', 'back_traj', 'adv_vert', 'adv_vflx', 'exch_data', 'exch_data.wait', 'global_sum', 'ordglb_sum', 'wrt_output', 'wait_for_async_io', 'physics', 'nwp_radiation', 'preradiaton', 'phys_acc_sync', 'satad', 'phys_u_v', 'nwp_turbulence', 'nwp_turbtrans', 'nwp_turbdiff', 'nwp_surface', 'nwp_microphysics', 'rediag_prog_vars', 'nwp_convection', 'sso', 'cloud_cover', 'radheat', 'nh_diagnostics', 'diagnose_pres_temp', 'model_init', 'compute_domain_decomp', 'compute_intp_coeffs', 'init_ext_data', 'init_icon', 'init_latbc', 'init_nwp_phy', 'upper_atmosphere', 'upatmo_construction', 'upatmo_destruction', 'write_restart', 'write_restart_io', 'write_restart_communication', 'optional_diagnostics_atmosphere'], call_nb=['86400', '432000', '518400', '864000', '864000', '864000', '864000', '17280

In [182]:
from dataclasses import dataclass, asdict
result_dict = asdict(results_dataclass)
print(result_dict)

{'name': ['integrate_nh', 'nh_solve', 'nh_solve.veltend', 'nh_solve.cellcomp', 'nh_solve.edgecomp', 'nh_solve.vnupd', 'nh_solve.vimpl', 'nh_solve.exch', 'nh_hdiff', 'transport', 'adv_horiz', 'adv_hflx', 'back_traj', 'adv_vert', 'adv_vflx', 'exch_data', 'exch_data.wait', 'global_sum', 'ordglb_sum', 'wrt_output', 'wait_for_async_io', 'physics', 'nwp_radiation', 'preradiaton', 'phys_acc_sync', 'satad', 'phys_u_v', 'nwp_turbulence', 'nwp_turbtrans', 'nwp_turbdiff', 'nwp_surface', 'nwp_microphysics', 'rediag_prog_vars', 'nwp_convection', 'sso', 'cloud_cover', 'radheat', 'nh_diagnostics', 'diagnose_pres_temp', 'model_init', 'compute_domain_decomp', 'compute_intp_coeffs', 'init_ext_data', 'init_icon', 'init_latbc', 'init_nwp_phy', 'upper_atmosphere', 'upatmo_construction', 'upatmo_destruction', 'write_restart', 'write_restart_io', 'write_restart_communication', 'optional_diagnostics_atmosphere'], 'call_nb': ['86400', '432000', '518400', '864000', '864000', '864000', '864000', '1728000', '8640

Now we need to get the time stamp and add it as an entry to the dictionary -- can legit take the last line which is when the thing passed the tests 

In [196]:
import re
from datetime import datetime

# Given format
date_format = "%a %d %b %Y %I:%M:%S %p %Z"
date_pattern = r'\b\w{3} \d{2} \w{3} \d{4} \d{2}:\d{2}:\d{2} [APM]{2} \w{4}\b'

def extract_and_convert(line):
    """Extract date strings and convert to datetime objects."""
    dates = re.findall(date_pattern, line)
    date_objects = [datetime.strptime(date, date_format) for date in dates]
    return date_objects

# Read the file and process each line
all_dates = []
with open('LOG.check.mch_bench_r19b07_icon2.run.o', 'r') as file:
    for line in file:
        all_dates.extend(extract_and_convert(line)) # Put this in the code at hte beginning 

# Sort and retrieve the latest date
all_dates.sort()
latest_date = all_dates[-1] if all_dates else None

print(latest_date) # Store this in the dictionary 


2023-10-15 13:18:54


In [199]:
result_dict["time_stamp"] = latest_date

[180215, 180294]
[' name                       # calls   t_min          t_avg          t_max          total min (s)   total max (s)   total avg (s)    ', ' ------------------------   -------   ------------   ------------   ------------   -------------   -------------   -------------    ']


In [66]:
from dataclasses import dataclass, field, fields
# Create a dataclass to save each line of the table of interest 
@dataclass
class MatchResultsICON: 
    name: list = field(default_factory=list)
    call_nb: list = field(default_factory=list)
    t_min: list = field(default_factory=list)
    min_rank:list = field(default_factory=list)
    t_avg: list = field(default_factory=list)
    t_max: list = field(default_factory=list)
    max_rank: list = field(default_factory=list)
    total_min: list = field(default_factory=list)
    total_min_rank:list = field(default_factory=list)
    total_max: list = field(default_factory=list)
    total_max_rank: list = field(default_factory=list)
    total_avg: list = field(default_factory=list)
    pe: list = field(default_factory=list)

    # To make it subscriptable
    def __getitem__(self, key):
        return getattr(self, key)


class Table:
    def __init__(self, lines):
        # Remove white spaces at the beginning and end of each line
        self.lines = [line.strip() for line in lines]

    def __str__(self):
        return "\n".join(self.lines)

 

attribute_names = [field.name for field in fields(MatchResultsICON)]

print(attribute_names)


header = log_file[start:start+1][0]
print(header)


['name', 'call_nb', 't_min', 'min_rank', 't_avg', 't_max', 'max_rank', 'total_min', 'total_min_rank', 'total_max', 'total_max_rank', 'total_avg', 'pe']
 name                       # calls   t_min          t_avg          t_max          total min (s)   total max (s)   total avg (s)    


In [68]:
import re
from typing import List
header_regex = r"name +.*calls.*"

with open("/users/clechart/probtest/util/icon/LOG.check.mch_bench_r19b07_icon2.run.o", "r", encoding="utf-8") as f:
    full_file = f.read()


def isolate_table(full_file: str, header_regex: str) -> List[Table]:
    """Mine the table from the log.

    Args:
        full_file (str): log file as string
        header_regex (str): regular expression pattern for table header

    Raises:
        ValueError: no information table found

    Returns:
        List[Table]: list of all tables available in the log

    """
    # Split file into lines and remove empty ones
    log_file = [e for e in full_file.split("\n") if e]

    # Find header lines
    header_lines = [i for i, e in enumerate(log_file) if re.search(header_regex, e)]
    
    # find end line (footer)
    pattern = r'-'*165
    footer_lines = [i for i, e in enumerate(log_file) if re.search(pattern, e)]

    if not header_lines or not footer_lines:
        logger.error("No headers or footers found in the log") 
        return

    tables = []

    for i in range(len(header_lines)):
        start = header_lines[i]
        # Check if we have a corresponding footer for this header
        end = next((f for f in footer_lines if f > start), None)
        if end is None:
            continue

        # Append table without headers
        table = Table(log_file[start+3:end])
        tables.append(table)

        # Remove this footer so it won't be used again
        footer_lines.remove(end)

    return tables


tables = isolate_table(full_file, header_regex)

In [70]:
print(tables[0])

total                             4               53m11s   [2]              53m11s         53m11s   [0]            3191.350    [2]                  3191.350    [0]                  3191.350    4
L integrate_nh                   86400         0.12085s   [2]            0.13805s        1.4055s   [3]            2980.473    [0]                  2984.872    [3]                  2981.987    4
L nh_solve                    432000        0.00909s   [0]            0.00977s       0.13249s   [3]            1034.114    [0]                  1064.997    [3]                  1055.603    4
L nh_solve.veltend         518400        0.00127s   [2]            0.00130s       0.03737s   [2]             168.866    [1]                   169.421    [3]                   169.106    4
L nh_solve.cellcomp        864000        0.00032s   [1]            0.00051s       0.00243s   [1]             110.164    [1]                   110.621    [0]                   110.367    4
L nh_solve.edgecomp        864000        0.0

In [71]:
import re

def process_line(line: str) -> list:
    """Remove white space and formatting in table line. 

    Args:
        line (str):individual log file line

    Returns:
        list: line elements as list
    """
    # Replace multiple consecutive whitespaces with a single '-'
    line = re.sub('[\s]+', '-', line)

    print(line)
    
    if line.startswith("L"):
        line = line.lstrip("L-")
        
    # Split the line into list
    elements = line.split('-')
    
    # Filter out unwanted elements
    elements = [elem for elem in elements if elem not in [" ", "(s)", "#", ""]]
    
    return elements



In [76]:
# FIXME we need to upload line by line instead of by table 
# so have to move the logic of process & upload to a level 
def store_tables(tables: list) -> MatchResultsICON:
    """Store table information in arranged manner.

    Args:
        tables (list): tables extracted from the log

    Returns:
        MatchResultsICON: ifnromation stored into a dataclass

    """
    results_dataclass = MatchResultsICON()
    # List of attribute names in the order they should be accessed
    attribute_names = [field.name for field in fields(MatchResultsICON)]
    
    for table in tables:
        for line in table.lines[1:]:
            print(line)
            print(len(line))
            # Check if the line has the right number of values for all attributes
            if len(line) == len(attribute_names):
                for i, attr_name in enumerate(attribute_names): 
                    try: 
                        value = float(line[i])
                        getattr(results_dataclass, attr_name).append(value)
                    except:
                        getattr(results_dataclass, attr_name).append(line[i])
                
                
            # If not the right length, skip the line
            else:
                continue

    return results_dataclass


results_dc = store_tables(tables)

L integrate_nh                   86400         0.12085s   [2]            0.13805s        1.4055s   [3]            2980.473    [0]                  2984.872    [3]                  2981.987    4
L nh_solve                    432000        0.00909s   [0]            0.00977s       0.13249s   [3]            1034.114    [0]                  1064.997    [3]                  1055.603    4
L nh_solve.veltend         518400        0.00127s   [2]            0.00130s       0.03737s   [2]             168.866    [1]                   169.421    [3]                   169.106    4
L nh_solve.cellcomp        864000        0.00032s   [1]            0.00051s       0.00243s   [1]             110.164    [1]                   110.621    [0]                   110.367    4
L nh_solve.edgecomp        864000        0.00052s   [2]            0.00078s       0.03303s   [0]             169.311    [1]                   169.388    [0]                   169.351    4
L nh_solve.vnupd           864000        0.00056s  

In [77]:
print(results_dc)

MatchResultsICON(name=[], call_nb=[], t_min=[], min_rank=[], t_avg=[], t_max=[], max_rank=[], total_min=[], total_min_rank=[], total_max=[], total_max_rank=[], total_avg=[], pe=[])


In [3]:
import re
def process_line(line: str) -> list:
    """Remove white space and formatting in table line.

    Args:
        line (str):individual log file line

    Returns:
        list: line elements as list
    """
    # Replace multiple consecutive whitespaces with a single '-'
    line = re.sub(r"[\s]+", "-", line)

    if line.startswith("L"):
        line = line.lstrip("L-")

    # Split the line into list
    elements = line.split("-")

    # Remove the units (s) at the end of the element
    elements = [elem.rstrip("s") if elem.endswith("s") else elem for elem in elements]

    # Filter out unwanted elements
    elements = [elem for elem in elements if elem not in [" ", "(s)", "#", ""]]

    return elements


In [9]:
process_line("Lsome/test/content")

['some/test/content']